<a href="https://colab.research.google.com/github/Luly7/CS-6480/blob/main/Roberta_good.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 0. Install Everything ---
!pip install -q transformers tensorflow
!pip install tf-keras
!pip install scikit-learn

# --- 1. Imports ---
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import mixed_precision
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from transformers import RobertaTokenizer, TFRobertaModel
from tqdm import tqdm

# --- 2. Enable Mixed Precision ---
mixed_precision.set_global_policy('mixed_float16')

# --- 3. Load your Data ---
csv_path = "/content/drive/MyDrive/CS6480/output_rt_fixed_smiles_unique.csv"
df = pd.read_csv(csv_path)
print(df.columns)

# --- 4. Load Tokenizer and Model ---
model_name = "distilroberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
transformer_model = TFRobertaModel.from_pretrained(model_name, from_pt=True, ignore_mismatched_sizes=True)
transformer_model.trainable = False

# --- 5. Define RT Prediction Model ---
class RTModel(tf.keras.Model):
    def __init__(self, transformer_model):
        super(RTModel, self).__init__()
        self.transformer = transformer_model
        self.dense = tf.keras.layers.Dense(1)

    def call(self, inputs, training=False):
        outputs = self.transformer(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            training=training
        ).last_hidden_state
        pooled_output = tf.reduce_mean(outputs, axis=1)
        rt_prediction = tf.cast(self.dense(pooled_output), dtype=tf.float32)
        return rt_prediction

# Instantiate model
rt_model = RTModel(transformer_model)

# --- 6. Tokenize SMILES ---
tokenized_inputs = tokenizer(
    list(df['fixed_smiles']),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="tf"
)

# --- 7. Create TensorFlow Dataset ---
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask']
    },
    tf.constant(df['rt'].values, dtype=tf.float32)
))

batch_size = 32
train_dataset = dataset.shuffle(buffer_size=len(df)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# --- 8. Checkpoint Setup ---
checkpoint_dir = "/content/drive/MyDrive/CS6480/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'ckpt-{epoch}.weights.h5'),
    save_weights_only=True,
    save_freq='epoch',
    verbose=1
)

# --- 9. Compile the Model ---
rt_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss='mse'
)

# --- 10. Train the Model ---
rt_model.fit(
    train_dataset,
    epochs=10,
    callbacks=[checkpoint_callback]
)

# --- 11. Save Final Model ---
final_model_dir = '/content/drive/MyDrive/CS6480/ROBERTA_1'

# === Safely check ===
if not os.path.exists(final_model_dir):
    os.makedirs(final_model_dir)
else:
    if not os.path.isdir(final_model_dir):
        raise NotADirectoryError(f"The path '{final_model_dir}' exists but is not a directory.")

# Now safe to save the model
final_model_path = os.path.join(final_model_dir, 'rt_model_final.keras')
rt_model.save(final_model_path)
print(f"✅ Final model saved at {final_model_path}")
final_model_path = os.path.join(final_model_dir, 'rt_model_final.keras')


# --- 12. Predict Retention Times ---
predict_dataset = tf.data.Dataset.from_tensor_slices({
    'input_ids': tokenized_inputs['input_ids'],
    'attention_mask': tokenized_inputs['attention_mask']
}).batch(batch_size)

predictions = rt_model.predict(predict_dataset)
predictions = predictions.flatten()

# --- 13. Calculate RMSE, MAE, R² ---
true_rt = df['rt'].values

rmse = np.sqrt(mean_squared_error(true_rt, predictions))
mae = mean_absolute_error(true_rt, predictions)
r2 = r2_score(true_rt, predictions)

print("\n✅ Final Evaluation Metrics:")
print(f"   RMSE: {rmse:.4f}")
print(f"   MAE:  {mae:.4f}")
print(f"   R²:   {r2:.4f}")

# --- 14. Save True vs Predicted into CSV ---
results_df = pd.DataFrame({
    'fixed_smiles': df['fixed_smiles'],
    'true_rt': true_rt,
    'predicted_rt': predictions
})

results_csv_path = os.path.join(final_model_dir, 'rt_predictions_vs_true.csv')
results_df.to_csv(results_csv_path, index=False)
print(f"✅ True vs Predicted Retention Times saved at {results_csv_path}")


Index(['rt', 'fixed_smiles'], dtype='object')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/10
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: 634366.3125
Epoch 1: saving model to /content/drive/MyDrive/CS6480/checkpoints/ckpt-1.weights.h5
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 334s 126ms/step - loss: 634353.3750
Epoch 2/10
2498/2499 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - loss: 512513.7188
Epoch 2: saving model to /content/drive/MyDrive/CS6480/checkpoints/ckpt-2.weights.h5
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 306s 122ms/step - loss: 512490.9688
Epoch 3/10
2498/2499 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - loss: 403895.3750
Epoch 3: saving model to /content/drive/MyDrive/CS6480/checkpoints/ckpt-3.weights.h5
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 322s 122ms/step - loss: 403877.4688
Epoch 4/10
2498/2499 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - loss: 314022.2500
Epoch 4: saving model to /content/drive/MyDrive/CS6480/checkpoints/ckpt-4.weights.h5
2499/2499 ━━━━━━━━━━━━━━━━━━━━ 321s 122ms/step - loss: 314005.8125
Epoch 5/10
2498/2499 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - loss: 235944.3906
Epoch 5: sav